#Set Up

In [1]:
!pip install crewai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 20.0 MB/s eta 0

In [2]:
from google.colab import userdata
import os
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

SERPER_API_KEY = userdata.get('SERPER_API_KEY')
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [3]:
from crewai import Agent, Task, Crew, LLM
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
llm = LLM(model="openai/gpt-3.5-turbo")

#Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    llm=llm,
    verbose=True
)

#Creating Tools

##crewAI Tools

In [6]:
!pip install crewai_tools==0.1.6

INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce 

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [9]:
search_tool

SerperDevTool(name='Search the internet', description="Search the internet(search_query: 'string') - A tool that can be used to semantic search a query from a txt's content.", args_schema=<class 'crewai_tools.tools.serper_dev_tool.serper_dev_tool.SerperDevToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7d253a4c1580>, search_url='https://google.serper.dev/search', n_results=None)

In [10]:
file_read_tool

FileReadTool(name="Read a file's content", description="Read a file's content(file_path: 'string') - A tool that can be used to read a file's content.", args_schema=<class 'crewai_tools.tools.file_read_tool.file_read_tool.FileReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7d253a4c1580>, file_path=None)

In [11]:
directory_read_tool

DirectoryReadTool(name='List files in directory', description="List files in directory() - A tool that can be used to list ./instructions's content.", args_schema=<class 'crewai_tools.tools.directory_read_tool.directory_read_tool.FixedDirectoryReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7d253a4c1580>, directory='./instructions')

##Custom Tool

In [12]:
from crewai_tools import BaseTool

In [13]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [14]:
sentiment_analysis_tool = SentimentAnalysisTool()

#Creating Tasks

In [15]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    #tools=[directory_read_tool, file_read_tool, search_tool],
    #tools = [directory_read_tool, file_read_tool, search_tool, sentiment_analysis_tool],
    #tools = [sentiment_analysis_tool],
    verbose=True,
    agent=sales_rep_agent,
)

In [16]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    #tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [17]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=True,
	  #memory=True
)

In [18]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

In [19]:
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 54050ea9-966d-48cc-a52f-d7eb8e280206                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1cfce94f-e697-473e-84fd-88fdb8f78eb9
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1cfce94f-e697-473e-84fd-88fdb8f78eb9
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🚀 Crew: crew
└── 📋 Task: 1cfce94f-e697-473e-84fd-88fdb8f78eb9
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Lead Sales Representative
## Final Answer: 
Subject: Elevating DeepLearningAI's Educational Excellence with Personalized Solutions

Dear Dr. Andrew Ng,

I hope this message finds you well. I am reaching out on behalf of CrewAI with great admiration for DeepLearningAI's recent achievements in advancing the field of online learning, particularly through the launch of your innovative deep learning certification program. Your commitment to staying at the forefront of industry trends while maintaining a focus on personalized education experiences truly sets DeepLearningAI apart as a leader in the AI education sector.

I am excited to share how our tailored training programs powered by AI can further enhance DeepLearningAI's mission to provide cutting-edge and personalized learning experiences. Our solutions offer customizable learning paths that adapt to individual student needs, ensuring higher engagement levels and improved knowledge retention. By integrating our platform seaml

🚀 Crew: crew
├── 📋 Task: 1cfce94f-e697-473e-84fd-88fdb8f78eb9
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: a983f415-b1e0-49c6-a3d5-e7ba6f62b18d
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1cfce94f-e697-473e-84fd-88fdb8f78eb9
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: a983f415-b1e0-49c6-a3d5-e7ba6f62b18d
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a983f415-b1e0-49c6-a3d5-e7ba6f62b18d                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 54050ea9-966d-48cc-a52f-d7eb8e280206                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
print(result)

Subject: Elevating DeepLearningAI's Educational Excellence with Personalized Solutions

Dear Dr. Andrew Ng,

I hope this message finds you well. I am reaching out on behalf of CrewAI with great admiration for DeepLearningAI's recent achievements in advancing the field of online learning, particularly through the launch of your innovative deep learning certification program. Your commitment to staying at the forefront of industry trends while maintaining a focus on personalized education experiences truly sets DeepLearningAI apart as a leader in the AI education sector.

I am excited to share how our tailored training programs powered by AI can further enhance DeepLearningAI's mission to provide cutting-edge and personalized learning experiences. Our solutions offer customizable learning paths that adapt to individual student needs, ensuring higher engagement levels and improved knowledge retention. By integrating our platform seamlessly with DeepLearningAI's infrastructure, we can elev

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

Subject: Elevating DeepLearningAI's Educational Excellence with Personalized Solutions

Dear Dr. Andrew Ng,

I hope this message finds you well. I am reaching out on behalf of CrewAI with great admiration for DeepLearningAI's recent achievements in advancing the field of online learning, particularly through the launch of your innovative deep learning certification program. Your commitment to staying at the forefront of industry trends while maintaining a focus on personalized education experiences truly sets DeepLearningAI apart as a leader in the AI education sector.

I am excited to share how our tailored training programs powered by AI can further enhance DeepLearningAI's mission to provide cutting-edge and personalized learning experiences. Our solutions offer customizable learning paths that adapt to individual student needs, ensuring higher engagement levels and improved knowledge retention. By integrating our platform seamlessly with DeepLearningAI's infrastructure, we can elevate the educational journey for each student, catering to their unique learning preferences and accelerating their mastery of complex AI concepts.

Additionally, as DeepLearningAI looks to expand its course offerings and reach a broader client base, our scalable infrastructure can effectively support your growth strategies. Our platform is designed to accommodate increased user traffic and diverse course offerings, providing the flexibility and scalability essential for meeting the evolving demands of your students and corporate clients.

Furthermore, leveraging our data-driven insights and analytics tools, DeepLearningAI can gain valuable information regarding student performance and engagement metrics. This empowers your team to optimize course content and delivery methods, ensuring that every learning experience is enriched with relevant and impactful educational material.

I believe that a partnership between DeepLearningAI and CrewAI can unlock new possibilities in the realm of online AI education, driving educational excellence and innovation forward. To illustrate the potential synergies between our organizations, I would be delighted to arrange a tailored demo to showcase how our solutions seamlessly integrate with DeepLearningAI's platform, delivering unparalleled personalized learning experiences.

Thank you for considering this opportunity to collaborate and enhance the educational landscape. I look forward to the possibility of working together to empower students and professionals worldwide with transformative AI education experiences.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Please customize the sender's name and contact details accordingly before sending the email to Dr. Andrew Ng.